In [3]:
import os
from pathlib import Path

import polars as pl

In [8]:
dfs = []
folder = Path(os.getcwd()).parent / 'data/merged_dataset/synthetic'
for f in os.listdir(folder):
    if f.endswith('.tsv') and not f.startswith('_'):
        dfs.append(pl.read_csv(folder / f, separator='\t', quote_char='\"', infer_schema=False))

In [9]:
dfs

[shape: (1_047_301, 5)
 ┌───────────────────────┬──────────────────────┬────────────────┬──────────────┬───────────────────┐
 │ input                 ┆ output               ┆ instruct       ┆ dataset_type ┆ dataloader_name   │
 │ ---                   ┆ ---                  ┆ ---            ┆ ---          ┆ ---               │
 │ str                   ┆ str                  ┆ str            ┆ str          ┆ str               │
 ╞═══════════════════════╪══════════════════════╪════════════════╪══════════════╪═══════════════════╡
 │ з фізики: Ерік        ┆ з фізики: Ерік       ┆ Використай     ┆ 1            ┆ UbertextV2Dataset │
 │ Корнелл, пКарл …      ┆ Корнелл, <g ed=…     ┆ наведені нижче ┆              ┆                   │
 │                       ┆                      ┆ інст…          ┆              ┆                   │
 │ 16 вересня вересня —: ┆ 16 <g ed="вересня"   ┆ Використай     ┆ 1            ┆ UbertextV2Dataset │
 │ початок …             ┆ et="F/Repet…         ┆ наведені 

In [10]:
merged = pl.concat(dfs)
shuffled_df = merged.sample(fraction=1, shuffle=True)

In [12]:
import math

train_df = shuffled_df.head(math.ceil(len(shuffled_df) * 0.60))
not_train_df = shuffled_df.tail(math.ceil(len(shuffled_df) * 0.40))
valid_df = not_train_df.head(math.ceil(len(not_train_df) * 0.40))
test_df = not_train_df.tail(math.ceil(len(not_train_df) * 0.60))


In [13]:
len(test_df), len(valid_df), len(not_train_df), len(train_df)

(430121, 286748, 716868, 1075301)

In [14]:
merged_folder = folder / 'merged'
train_df.write_csv(merged_folder / 'train.csv', separator='\t', quote_char='\"')
valid_df.write_csv(merged_folder / 'valid.csv', separator='\t', quote_char='\"')
test_df.write_csv(merged_folder / 'test.csv', separator='\t', quote_char='\"')
